In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, BatchNormalization, Activation
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping
from keras.saving import register_keras_serializable

# Register custom functions (loss functions in this case)
@register_keras_serializable()
def mse(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

@register_keras_serializable()
def mae(y_true, y_pred):
    return tf.reduce_mean(tf.abs(y_true - y_pred))

# Define paths to your dataset
data_dir = "..//cv_p3_images_split"
train_gray_dir = os.path.join(data_dir, "train/grayscale")
train_color_dir = os.path.join(data_dir, "train/colored")
val_gray_dir = os.path.join(data_dir, "validation/grayscale")
val_color_dir = os.path.join(data_dir, "validation/colored")

# Image dimensions for resizing
IMG_HEIGHT, IMG_WIDTH = 256, 256  # Resize all images to 256x256

# Utility function to preprocess images (loading and resizing)
def preprocess_image(image_path, target_size):
    image = load_img(image_path, target_size=target_size, color_mode="rgb")
    image = img_to_array(image) / 255.0  # Normalize to [0, 1]
    return image

# Function to load a dataset from a folder (grayscale and color images)
def load_dataset(gray_folder, color_folder, target_size, fraction=1.0):
    gray_images = []
    color_images = []
    
    filenames = os.listdir(gray_folder)
    if fraction < 1.0:
        filenames = random.sample(filenames, int(len(filenames) * fraction))  # Sample fraction of filenames
    
    for filename in filenames:
        gray_path = os.path.join(gray_folder, filename)
        color_path = os.path.join(color_folder, filename)
        gray_images.append(preprocess_image(gray_path, target_size)[..., 0:1])  # Extract grayscale channel
        color_images.append(preprocess_image(color_path, target_size))
    
    return np.array(gray_images), np.array(color_images)

# Load train and validation datasets
train_gray, train_color = load_dataset(train_gray_dir, train_color_dir, (IMG_HEIGHT, IMG_WIDTH), 0.25)
val_gray, val_color = load_dataset(val_gray_dir, val_color_dir, (IMG_HEIGHT, IMG_WIDTH), 0.25)

# Define the colorization model architecture
def build_model(loss="mse"):
    inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 1))

    # Encoder
    x = Conv2D(32, (3, 3), padding="same", strides=2)(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(64, (3, 3), padding="same", strides=2)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # Decoder
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = UpSampling2D((2, 2))(x)
    x = Conv2D(3, (3, 3), padding="same")(x)
    outputs = Activation("sigmoid")(x)

    return Model(inputs, outputs)

# Define cross-validation setup with KFold (5 folds)
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Create an EarlyStopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Perform cross-validation and train the model
for fold, (train_index, val_index) in enumerate(kf.split(train_gray)):
    print(f'Fold {fold + 1}/{num_folds}')

    # Split the data into training and validation for this fold
    fold_train_gray, fold_val_gray = train_gray[train_index], train_gray[val_index]
    fold_train_color, fold_val_color = train_color[train_index], train_color[val_index]

    # Build and compile the model
    model = build_model()
    
    # Compile the model using RMSProp optimizer
    model.compile(
        optimizer=RMSprop(learning_rate=0.001),  # RMSProp optimizer with learning rate 0.001
        loss="mse",  # Mean squared error loss function
        metrics=['accuracy']  # You can add other metrics if needed
    )

    # Train the model
    history = model.fit(
        fold_train_gray, fold_train_color,
        validation_data=(fold_val_gray, fold_val_color),
        epochs=50,
        batch_size=16,
        callbacks=[early_stopping]  # Use early stopping to prevent overfitting
    )

    # Save the model after training
    model.save(f'colorization_model_fold_{fold + 1}.h5')

    # Optionally, you can save the training history and plot results
    # Save the training and validation loss plot
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Training and Validation Loss - Fold {fold + 1}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'raport_sources/training_validation_loss_fold_{fold + 1}.png')  # Save plot
    plt.close()

    # Optionally, you can save predictions for further analysis
    # predictions = model.predict(fold_val_gray)
    # Save or process the predictions as required.

print("Cross-validation completed.")
